In [7]:
# df_static

# test

# Machine Learning Regression

In [8]:
df_train = pd.concat([
    df_static[df_static["formation_e"].notnull()],
    df_oqmd,
    ])

df_test = df_static

/home/raul_desktop/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [9]:
list(df_test)


['atoms', 'formation_e', 'id']

In [10]:
test_atoms = df_test["atoms"].tolist()

VFG = VoronoiFingerprintGenerator(test_atoms)
df_vor = VFG.generate()
test_features = df_vor.values
    
pred_list = []

for i in range(2, len(df_train), 7):
    print(i)

    i_ind = i

    df_train_i = df_train[0:i_ind]

    train_atoms = df_train_i["atoms"].tolist()

    generator = FeatureGenerator()


#     generator.normalize_features(
#         train_candidates=train_atoms,
#         test_candidates=test_atoms,
#         )

#     train_features = generator.return_vec(
#         train_atoms,
#         [
#             generator.eigenspectrum_vec,
#             generator.composition_vec,
#             ],
#         )

#     test_features = generator.return_vec(
#         test_atoms,
#         [
#             generator.eigenspectrum_vec,
#             generator.composition_vec,
#             ]
#         )


    VFG = VoronoiFingerprintGenerator(df_train_i["atoms"].tolist())
    df_vor = VFG.generate()
    train_features = df_vor.values





    kdict = [{'type': 'gaussian', 'width': 1., 'scaling': 1., 'dimension': 'single'}]

    train_targets = df_train_i["formation_e"].tolist()
    # test_targets = df_test["target"].tolist()

    gp = GaussianProcess(
        train_fp=train_features,
        train_target=train_targets,
        kernel_list=kdict,
        regularization=1e-2,
        optimize_hyperparameters=True,
        scale_data=True,
        )

    pred = gp.predict(
        test_fp=test_features,
        uncertainty=True,
        )

    pred_list.append(
        {
            "num_training": len(df_train_i),
            "prediction": pred["prediction"],
            "prediction_sorted": np.sort(pred["prediction"]),
            "id": df_test["id"],
            "uncertainty": pred["uncertainty"],
            "uncertainty_with_reg": pred["uncertainty_with_reg"],
            }
        )
    

Generate Voronoi fingerprint of 259 structures
2
Generate Voronoi fingerprint of 2 structures
9
Generate Voronoi fingerprint of 9 structures
16
Generate Voronoi fingerprint of 16 structures
23
Generate Voronoi fingerprint of 23 structures
30
Generate Voronoi fingerprint of 30 structures
37
Generate Voronoi fingerprint of 37 structures


# Plotting with my analysis code

In [11]:
import os
import sys

sys.path.insert(0, os.path.join(
        os.environ["PROJ_irox"],
        "workflow",
        "ml_convergence_plots",
        ))

from methods import process_data_for_plot, get_layout

In [12]:
save_dir = "__temp__"

In [13]:
data_final = []

for i in pred_list:   
    df_i = pd.DataFrame(i)
    df_i["ind"] = [i for i in range(len(df_i))]

    df_tmp_i = df_i.sort_values("prediction")
    df_tmp_i["ind_new"] = [i for i in range(len(df_tmp_i))]

    df_tmp_i["ind_old"] = df_tmp_i["ind"]
    df_tmp_i["ind"] = df_tmp_i["ind_new"]

    data1 = process_data_for_plot(
        df_tmp_i,
        color0="red",
        color1="black",
        name=str(df_i["num_training"].iloc[0]),
        energy_key="prediction",
        uncertainty_key="uncertainty",
        )
    data_final += data1

In [14]:
layout = get_layout()
# layout["xaxis"]["range"]
layout["yaxis"]["range"] = [-1.5, 2.]

layout["showlegend"] = True
fig = dict(data=data_final, layout=layout)
py.iplot(fig,
    filename=os.path.join(
        save_dir,
        "ml_regression_test",
        "190317_iro3_ml_training",
        ),
    )

# Plotting

In [15]:
# data = []
# for pred_i in pred_list:
#     x_array = [i for i in range(len(pred_i["prediction"]))]
    
#     trace = go.Scatter(
#         x=x_array,
# #         y=pred_i["prediction"],
#         y=pred_i["prediction_sorted"],
# #         y=pred_i["prediction"],
#         mode='lines+markers',
#         name=str(pred_i["num_training"]),
#     )

#     data.append(trace)

# # Plot and embed in ipython notebook!
# py.iplot(data, filename='__temp__/basic-scatter')

In [16]:
# VFG = VoronoiFingerprintGenerator(df_train["atoms"].tolist())

# df_vor = VFG.generate()

# features = df_vor.values

# len(df_vor)

# df_vor.values.shape

# train_features.shape

# len(df_train)

In [17]:
# pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', None)

# pd.DataFrame(
#     test_features,
#     columns=generator.return_names(
#         [
#             generator.eigenspectrum_vec,
#             generator.composition_vec,
#             ]
#         ),
#     )

In [18]:
# pred_list_new = []
# for i in pred_list:
#     pred_sorted_i = np.sort(i["prediction"])
#     pred_list_new.append(pred_sorted_i)